In [ ]:
# Cross-Encoder Reranker for Ontology Disambiguation
# Training script for Google Colab

# ============================================================================
# 1. INSTALLATION
# ============================================================================

!pip install -q sentence-transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.1 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installe

In [ ]:
!pip install --upgrade "libraft-cu12==25.6.*" "pylibraft-cu12==25.6.*" "rmm-cu12==25.6.*"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 85.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 876.0/876.0 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.7 MB/s eta 0:00:00
  Attempting uninstall: libraft-cu12
    Found existing installation: libraft-cu12 25.2.0
    Uninstalling libraft-cu12-25.2.0:
      Successfully uninstalled libraft-cu12-25.2.0
  Attempting uninstall: rmm-cu12
    Found existing installation: rmm-cu12 25.2.0
    Uninstalling rmm-cu12-25.2.0:
      Successfully uninstalled rmm-cu12-25.2.0
  Attempting uninstall: pylibraft-cu12
    Found existing installation: pylibraft-cu12 25.2.0
    Uninstalling pylibraft-cu12-25.2.0:
      Successfully uninstalled pylibraft-cu12-25.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the 

In [ ]:
# ============================================================================
# 2. IMPORTS
# ============================================================================

import json
import os
import torch
from datasets import Dataset
from sentence_transformers.cross_encoder import CrossEncoder, CrossEncoderTrainer
from sentence_transformers.cross_encoder import losses
from sentence_transformers.cross_encoder.training_args import CrossEncoderTrainingArguments
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator
from typing import Dict, List

# ============================================================================
# 3. LOAD DATA
# ============================================================================

def load_jsonl(file_path: str) -> List[Dict]:
    """Load JSONL file into a list of dictionaries."""
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

# Load datasets
print("Loading datasets...")
train_data = load_jsonl('/kaggle/input/reranker-training-data/train.jsonl')
dev_data = load_jsonl('/kaggle/input/reranker-training-data/dev.jsonl')
test_data = load_jsonl('/kaggle/input/reranker-training-data/test.jsonl')

print(f"Train samples: {len(train_data)}")
print(f"Dev samples: {len(dev_data)}")
print(f"Test samples: {len(test_data)}")

# Display sample
print("\nSample training example:")
print(json.dumps(train_data[0], indent=2))

# ============================================================================
# 4. PREPARE DATASETS FOR CROSS-ENCODER
# ============================================================================

def prepare_cross_encoder_dataset(data: List[Dict]) -> Dataset:
    """
    Convert raw data to cross-encoder format.

    Each example should have:
    - sentence1: query
    - sentence2: candidate
    - label: binary label (0 or 1)
    """
    prepared_data = {
        'sentence1': [],
        'sentence2': [],
        'label': []
    }

    for item in data:
        prepared_data['sentence1'].append(item['query'])
        prepared_data['sentence2'].append(item['candidate'])
        prepared_data['label'].append(float(item['label']))  # Ensure float for BCE loss

    return Dataset.from_dict(prepared_data)

# Prepare datasets
print("\nPreparing datasets for cross-encoder training...")
train_dataset = prepare_cross_encoder_dataset(train_data)
dev_dataset = prepare_cross_encoder_dataset(dev_data)
test_dataset = prepare_cross_encoder_dataset(test_data)

print(f"Prepared train dataset: {len(train_dataset)} samples")
print(f"Prepared dev dataset: {len(dev_dataset)} samples")
print(f"Prepared test dataset: {len(test_dataset)} samples")

# Display prepared sample
print("\nPrepared sample:")
print(train_dataset[0])

Loading datasets...
Train samples: 2401485
Dev samples: 132647
Test samples: 134290

Sample training example:
{
  "query": "cell_type: cDC1; tissue: tonsil; organism: Homo sapiens",
  "candidate": "label: tonsil germinal center B cell; definition: Any germinal center B cell that is part of a tonsil.",
  "candidate_id": "CL:2000006",
  "correct_id": "CL:0000990",
  "label": 0,
  "retrieval_score": 1.0,
  "retrieval_rank": 0,
  "example_type": "hard_negative"
}

Preparing datasets for cross-encoder training...
Prepared train dataset: 2401485 samples
Prepared dev dataset: 132647 samples
Prepared test dataset: 134290 samples

Prepared sample:
{'sentence1': 'cell_type: cDC1; tissue: tonsil; organism: Homo sapiens', 'sentence2': 'label: tonsil germinal center B cell; definition: Any germinal center B cell that is part of a tonsil.', 'label': 0.0}


In [ ]:
# ============================================================================
# 5. INITIALIZE MODEL
# ============================================================================
print("\nInitializing CrossEncoder model...")
model = CrossEncoder(
    "bioformers/bioformer-16L",
    num_labels=1,  # Binary classification (relevance score)
    max_length=512,  # Maximum sequence length
    device='cuda' if torch.cuda.is_available() else 'cpu'
)
print(f"Model loaded on device: {model.device}")
print(f"Model max length: {model.max_length}")

# ============================================================================
# 6. SETUP LOSS FUNCTION
# ============================================================================
# Binary Cross Entropy Loss for binary relevance prediction
pos_weight = torch.tensor([5.0])  # Weight positives 5x more
loss = losses.BinaryCrossEntropyLoss(model, pos_weight=pos_weight)
print(f"\nUsing BinaryCrossEntropyLoss with pos_weight={pos_weight.item()}")

# ============================================================================
# 7. SETUP EVALUATOR
# ============================================================================
# Create evaluator for binary classification
evaluator = CEBinaryClassificationEvaluator(
    sentence_pairs=list(zip(dev_dataset['sentence1'], dev_dataset['sentence2'])),
    labels=dev_dataset['label'],
    name='dev'
)
print("Evaluator configured for development set")

# ============================================================================
# 8. CONFIGURE TRAINING ARGUMENTS
# ============================================================================
training_args = CrossEncoderTrainingArguments(
    output_dir="./results-new",

    # Training hyperparameters
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_ratio=0.1,

    # Evaluation and saving
    eval_strategy='steps',
    eval_steps=500,
    save_strategy='steps',
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='dev_f1',

    # Optimization
    fp16=torch.cuda.is_available(),  # Use mixed precision if GPU available
    gradient_accumulation_steps=1,
    max_grad_norm=1.0,

    # Logging
    logging_dir='./logs',
    logging_steps=100,
    logging_first_step=True,
    report_to='none',  # Change to 'wandb' or 'tensorboard' if needed

    # Other settings
    seed=42,
    dataloader_drop_last=False,
)

print("\nTraining arguments configured:")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Batch size: {training_args.per_device_train_batch_size}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  FP16: {training_args.fp16}")


Initializing CrossEncoder model...


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bioformers/bioformer-16L and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Model loaded on device: cuda:0
Model max length: 512

Using BinaryCrossEntropyLoss with pos_weight=5.0


/tmp/ipykernel_48/2773534198.py:26: DeprecationWarning: This evaluator has been deprecated in favor of the more general CrossEncoderClassificationEvaluator. Please use CrossEncoderClassificationEvaluator instead, which supports both binary and multi-class evaluation. It accepts approximately the same inputs as this evaluator.
  evaluator = CEBinaryClassificationEvaluator(


Evaluator configured for development set

Training arguments configured:
  Epochs: 3
  Batch size: 32
  Learning rate: 2e-05
  FP16: True


In [ ]:
# ============================================================================
# 9. CHECKPOINT CONFIGURATION (RESUME SUPPORT)
# ============================================================================
import os
import torch

checkpoint_dir = "/kaggle/input/checkpoint-85500/"
print(f"\nCheckpoint directory: {checkpoint_dir}")

resume_from_checkpoint = None

# Direct checkpoint file detection (flat checkpoint structure)
model_file = os.path.join(checkpoint_dir, "pytorch_model.bin")
safetensors_file = os.path.join(checkpoint_dir, "model.safetensors")

if os.path.exists(model_file) or os.path.exists(safetensors_file):
    resume_from_checkpoint = checkpoint_dir
    print(f"✓ Found checkpoint: {checkpoint_dir}")
    print("  Will resume training from this checkpoint.")
else:ci
    print("ℹ️  No existing checkpoints found. Starting fresh training.")

# Enable RNG loading for HF checkpoints - GET TRUE ORIGINAL
if resume_from_checkpoint:
    # Get the TRUE original torch.load from torch.serialization
    # This bypasses any previous patches that might be in memory
    import torch.serialization
    _true_original_load = torch.serialization.load

    # Create wrapper that always uses the true original
    def custom_torch_load(*args, **kwargs):
        if 'weights_only' not in kwargs:
            kwargs['weights_only'] = False
        return _true_original_load(*args, **kwargs)

    torch.load = custom_torch_load
    print("✓ Configured torch.load to support RNG state loading")

# ============================================================================
# 10. CUSTOM EARLY STOPPING CALLBACK
# ============================================================================
from transformers import TrainerCallback

class ManualEarlyStoppingCallback(TrainerCallback):
    """
    Custom early stopping callback compatible with CrossEncoderTrainer.
    Monitors dev_f1 metric and stops training if no improvement.
    """
    def __init__(self, patience=3, threshold=0.01, metric_name='dev_f1'):
        self.patience = patience
        self.threshold = threshold
        self.metric_name = metric_name
        self.best_metric = None
        self.patience_counter = 0
        self.should_stop = False

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        """Called after evaluation"""
        if metrics is None:
            return control

        current_metric = metrics.get(self.metric_name)

        if current_metric is None:
            return control

        # First evaluation
        if self.best_metric is None:
            self.best_metric = current_metric
            print(f"\n📊 Initial {self.metric_name}: {current_metric:.4f}")
            return control

        # Check for improvement
        improvement = current_metric - self.best_metric

        if improvement > self.threshold:
            # Improved significantly
            self.best_metric = current_metric
            self.patience_counter = 0
            print(f"\n✓ {self.metric_name} improved to {current_metric:.4f} (+{improvement:.4f})")
        else:
            # No significant improvement
            self.patience_counter += 1
            print(f"\n⚠️  No improvement in {self.metric_name}: {current_metric:.4f} "
                  f"(best: {self.best_metric:.4f}, patience: {self.patience_counter}/{self.patience})")

            if self.patience_counter >= self.patience:
                print(f"\n🛑 Early stopping triggered! No improvement for {self.patience} evaluations.")
                control.should_training_stop = True
                self.should_stop = True

        return control

    def state(self):
        """Return callback state for checkpointing"""
        return {
            'best_metric': self.best_metric,
            'patience_counter': self.patience_counter,
        }

    def load_state(self, state):
        """Load callback state from checkpoint"""
        self.best_metric = state.get('best_metric')
        self.patience_counter = state.get('patience_counter', 0)
        print(f"✓ Restored early stopping state: best_metric={self.best_metric}, patience_counter={self.patience_counter}")

# Initialize early stopping callback
early_stopping = ManualEarlyStoppingCallback(
    patience=1,
    threshold=0.0001,
    metric_name='dev_f1'
)

print("✓ Manual early stopping callback initialized")
print(f"  Patience: 1 evaluations")
print(f"  Threshold: 0.0001")
print(f"  Metric: dev_f1")

# ============================================================================
# 11. INITIALIZE TRAINER
# ============================================================================
trainer = CrossEncoderTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    loss=loss,
    evaluator=evaluator,
    callbacks=[early_stopping]
)

print("\nTrainer initialized successfully")

# Restore early stopping state if resuming
if resume_from_checkpoint:
    # Try to load early stopping state from checkpoint
    early_stopping_state_file = os.path.join(checkpoint_dir, "early_stopping_state.json")
    if os.path.exists(early_stopping_state_file):
        import json
        with open(early_stopping_state_file, 'r') as f:
            es_state = json.load(f)
        early_stopping.load_state(es_state)
    else:
        print("ℹ️  No early stopping state found in checkpoint, starting fresh")

# ============================================================================
# 12. TRAIN MODEL (WITH CHECKPOINT RESUME)
# ============================================================================
print("\n" + "="*80)
if resume_from_checkpoint:
    print("RESUMING TRAINING FROM CHECKPOINT")
    print(f"Checkpoint: {resume_from_checkpoint}")
else:
    print("STARTING TRAINING")
print("="*80 + "\n")

trainer.train(resume_from_checkpoint=resume_from_checkpoint)

print("\n" + "="*80)
print("TRAINING COMPLETED")
print("="*80 + "\n")

# Save early stopping state for future resume
import json
es_state_file = os.path.join(training_args.output_dir, "early_stopping_state.json")
with open(es_state_file, 'w') as f:
    json.dump(early_stopping.state(), f)
print(f"✓ Early stopping state saved to {es_state_file}")

# ============================================================================
# 13. EVALUATE ON TEST SET
# ============================================================================
print("Evaluating on test set...")

test_evaluator = CEBinaryClassificationEvaluator(
    sentence_pairs=list(zip(test_dataset['sentence1'], test_dataset['sentence2'])),
    labels=test_dataset['label'],
    name='test'
)

test_results = test_evaluator(model)

print("\nTest Set Results:")
for metric, value in test_results.items():
    print(f"  {metric}: {value:.4f}")

# ============================================================================
# 14. SAVE MODEL
# ============================================================================
output_path = "./ontology-reranker"
model.save_pretrained(output_path)
print(f"\nModel saved to: {output_path}")


Checkpoint directory: /kaggle/input/checkpoint-85500/
✓ Found checkpoint: /kaggle/input/checkpoint-85500/
  Will resume training from this checkpoint.
✓ Configured torch.load to support RNG state loading
✓ Manual early stopping callback initialized
  Patience: 1 evaluations
  Threshold: 0.0001
  Metric: dev_f1

Trainer initialized successfully
ℹ️  No early stopping state found in checkpoint, starting fresh

RESUMING TRAINING FROM CHECKPOINT
Checkpoint: /kaggle/input/checkpoint-85500/



Step,Training Loss,Validation Loss,Dev Accuracy,Dev Accuracy Threshold,Dev F1,Dev F1 Threshold,Dev Precision,Dev Recall,Dev Average Precision
86000,0.183400,0.230241,0.974685,0.975106,0.819161,0.951832,0.801594,0.837515,0.886336
86500,0.169300,0.208405,0.975047,0.960146,0.817809,0.924933,0.795648,0.841240,0.888710
87000,0.152500,0.234804,0.975559,0.957142,0.821792,0.806979,0.808590,0.835433,0.891693
87500,0.164600,0.232705,0.975484,0.969608,0.820313,0.947480,0.820313,0.820313,0.892611
88000,0.182300,0.241580,0.975597,0.982167,0.819810,0.816658,0.789607,0.852416,0.890503
88500,0.207000,0.220260,0.975544,0.969271,0.823629,0.930022,0.799423,0.849348,0.888866
89000,0.161100,0.304276,0.975702,0.967635,0.824336,0.608536,0.800268,0.849896,0.885386
89500,0.169700,0.225957,0.975423,0.957369,0.822275,0.845350,0.808601,0.836419,0.895673
90000,0.160800,0.223263,0.975544,0.960353,0.815986,0.903579,0.812134,0.819875,0.889527


In [ ]:
# ============================================================================
# 14. INFERENCE EXAMPLE
# ============================================================================

print("\n" + "="*80)
print("INFERENCE EXAMPLE")
print("="*80 + "\n")

# Load saved model (optional - for demonstration)
# loaded_model = CrossEncoder(output_path)

# Example inference
example_query = "cell_type: C_BEST4; tissue: descending colon; organism: Homo sapiens"
example_candidates = [
    "label: smooth muscle fiber of descending colon; synonyms: non-striated muscle fiber of descending colon; definition: A smooth muscle cell that is part of the descending colon.",
    "label: smooth muscle cell of colon; synonyms: non-striated muscle fiber of colon; definition: A smooth muscle cell that is part of the colon.",
    "label: epithelial cell of colon; synonyms: colon epithelial cell; definition: An epithelial cell that is part of the colon."
]

print("Query:")
print(f"  {example_query}\n")

print("Ranking candidates...")
# Create pairs
pairs = [(example_query, candidate) for candidate in example_candidates]

# Get predictions
scores = model.predict(pairs)

# Rank by score
ranked_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)

print("\nRanked Results:")
for rank, idx in enumerate(ranked_indices, 1):
    print(f"\n{rank}. Score: {scores[idx]:.4f}")
    print(f"   Candidate: {example_candidates[idx][:100]}...")

# ============================================================================
# 15. BATCH RANKING EXAMPLE
# ============================================================================

print("\n" + "="*80)
print("BATCH RANKING WITH model.rank()")
print("="*80 + "\n")

# Using the convenient rank() method
ranked_results = model.rank(
    example_query,
    example_candidates,
    return_documents=True,
    top_k=3
)

print("Top 3 ranked results using model.rank():")
for result in ranked_results:
    print(f"\nRank: {result['corpus_id'] + 1}")
    print(f"Score: {result['score']:.4f}")
    print(f"Text: {result['text'][:100]}...")

print("\n" + "="*80)
print("SCRIPT COMPLETED SUCCESSFULLY")
print("="*80)

In [ ]:
# ============================================================================
# INFERENCE USING CHECKPOINT MODEL
# ============================================================================

import os
import torch
from sentence_transformers.cross_encoder import CrossEncoder

checkpoint_path = "/kaggle/input/checkpoint-78500/"

print("\n" + "="*80)
print("INFERENCE EXAMPLE")
print("="*80 + "\n")

# Load saved model from checkpoint
loaded_model = CrossEncoder(checkpoint_path, device='cuda' if torch.cuda.is_available() else 'cpu')

# Example inference
example_query = "cell_type: C_BEST4; tissue: descending colon; organism: Homo sapiens"
example_candidates = [
    "label: smooth muscle fiber of descending colon; synonyms: non-striated muscle fiber of descending colon; definition: A smooth muscle cell that is part of the descending colon.",
    "label: smooth muscle cell of colon; synonyms: non-striated muscle fiber of colon; definition: A smooth muscle cell that is part of the colon.",
    "label: epithelial cell of colon; synonyms: colon epithelial cell; definition: An epithelial cell that is part of the colon."
]

print("Query:")
print(f"  {example_query}\n")

print("Ranking candidates...")
# Create pairs
pairs = [(example_query, candidate) for candidate in example_candidates]

# Get predictions (raw logits)
raw_scores = loaded_model.predict(pairs)

# Convert to probabilities (0-1) using sigmoid
import torch.nn.functional as F
scores = torch.sigmoid(torch.tensor(raw_scores)).numpy()

# Rank by score
ranked_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)

print("\nRanked Results:")
for rank, idx in enumerate(ranked_indices, 1):
    print(f"\n{rank}. Probability: {scores[idx]:.4f}")
    print(f"   Candidate: {example_candidates[idx]}")

# ============================================================================
# BATCH RANKING EXAMPLE
# ============================================================================

print("\n" + "="*80)
print("BATCH RANKING WITH model.rank()")
print("="*80 + "\n")

# Using the convenient rank() method
ranked_results = loaded_model.rank(
    example_query,
    example_candidates,
    return_documents=True,
    top_k=3
)

print("Top 3 ranked results using model.rank():")
for result in ranked_results:
    # Convert score to probability if needed
    prob = torch.sigmoid(torch.tensor(result['score'])).item()
    print(f"\nRank: {result['corpus_id'] + 1}")
    print(f"Probability: {prob:.4f}")
    print(f"Text: {result['text']}")

print("\n" + "="*80)
print("SCRIPT COMPLETED SUCCESSFULLY")
print("="*80)



INFERENCE EXAMPLE

Query:
  cell_type: C_BEST4; tissue: descending colon; organism: Homo sapiens

Ranking candidates...

Ranked Results:

1. Probability: 0.5016
   Candidate: label: epithelial cell of colon; synonyms: colon epithelial cell; definition: An epithelial cell that is part of the colon.

2. Probability: 0.5001
   Candidate: label: smooth muscle fiber of descending colon; synonyms: non-striated muscle fiber of descending colon; definition: A smooth muscle cell that is part of the descending colon.

3. Probability: 0.5001
   Candidate: label: smooth muscle cell of colon; synonyms: non-striated muscle fiber of colon; definition: A smooth muscle cell that is part of the colon.

BATCH RANKING WITH model.rank()

Top 3 ranked results using model.rank():

Rank: 3
Probability: 0.5016
Text: label: epithelial cell of colon; synonyms: colon epithelial cell; definition: An epithelial cell that is part of the colon.

Rank: 1
Probability: 0.5001
Text: label: smooth muscle fiber of descend